# Lab / HomeWork
## [Artificial Neural Network (ANN)]
## Feedforward Neural Network (FFNN)

In [103]:
# imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from collections import Counter


class ANN:

    def __init__(self, file_path, v_neuron, w_neuron):
        self.data = pd.read_csv(file_path, header = None, sep=" ")

        self.v_neuron = v_neuron
        self.w_neuron = w_neuron

        self.matrix_y = self.y_matrix(self.data[2])

        self.matrix_x = self.input_matrix_x()
        self.matrix_x_extended = self.x_1_matrix()

        self.matrix_v = self.v_matrix()
        self.matrix_x_extended_2 = self.x_2_matrix()

        self.matrix_f = self.f_matrix()
        self.matrix_f_extended = self.f_1_matrix()

        self.matrix_w = self.w_matrix()
        self.matrix_f_extended_2 = self.f_2_matrix()

        self.matrix_g = self.g_matrix()

        self.error_history = []

$$[x]_\times = \begin{pmatrix} 0 & −x3 & x2 \\ x3 & 0 & −x1 \\ −x2 & x1 & 0 \end{pmatrix}$$

In [104]:
class ANN(ANN):
    def y_matrix(self, y_data):
        labels_y = [x for x in list(Counter(y_data).keys())]
        labels_y.sort()
        number_of_labels = len(Counter(y_data).keys())
        
        matrix_y_init = np.zeros((len(self.data), number_of_labels))
        for i in range(len(self.data)):
            for k in range(number_of_labels):
                if y_data[i] == labels_y[k]:
                    matrix_y_init[i][k] = 1
        return matrix_y_init

    def input_matrix_x(self):
        x = np.array([self.data[0]]).transpose()
        for i in range(1, len(self.data.columns) - 1):
            b = np.array([self.data[i]]).transpose()
            x = np.concatenate((x, b), axis=1)
        print(x)
        print()
        return x

    def x_1_matrix(self):
        ones = np.ones(((np.size(self.matrix_x, 0)), 1))
        return np.append(ones, self.matrix_x, axis=0)

    def v_matrix(self):
        return np.random.rand(np.size(self.matrix_x_extended, 1), self.v_neuron)

    def x_2_matrix(self):
        return self.matrix_x_extended.dot(self.matrix_v)

    def f_matrix(self):
        return 1 / (1 + np.exp(-self.matrix_x_extended_2))

    def f_1_matrix(self):
        ones = np.ones((np.size(self.matrix_f, 0), 1))
        return np.append(ones, self.matrix_f, axis=1)

    def w_matrix(self):
        return np.random.rand(self.v_neuron+1, self.w_neuron)

    def f_2_matrix(self):
        return self.matrix_f_extended.dot(self.matrix_w)

    def g_matrix(self):
        return 1 / (1 + np.exp(-self.matrix_f_extended_2))

    def sum_squared_errors_calculation(self):
        squares = (self.matrix_g - self.matrix_y) ** 2
        return np.sum(squares) / 2

    def w_correction(self, alpha_w):
        w_matrix_new = np.zeros((np.size(self.matrix_w, 0), np.size(self.matrix_w, 1)))
        # w of size (k+1)xJ
        for k in range(np.size(self.matrix_w, 0)):
            for j in range(np.size(self.matrix_w, 1)):
                w = self.matrix_w[k][j]
                err = 0
                for i in range(np.size(self.matrix_g, 1)):
                    g = self.matrix_g[i][j]
                    y = self.matrix_y[i][j]
                    f_ext = self.matrix_f_extended[i][k]

                    err += (g - y) * g * (1 - g) * f_ext
                w_matrix_new[k][j] = w - (alpha_w * err)
        return w_matrix_new

In [105]:
class ANN(ANN):
    def v_correction(self, alpha_v):
        v_matrix_new = np.zeros((np.size(self.matrix_v, 0), np.size(self.matrix_v, 1)))
        for n in range(np.size(self.matrix_x_extended, 1)):
            for k in range(np.size(self.matrix_v, 1)):
                v = self.matrix_v[n][k]
                err = 0
                for i in range(np.size(self.matrix_g, 0)):
                    f = self.matrix_f[i][k]
                    x_ext = self.matrix_x_extended[i][n]

                    for j in range(np.size(self.matrix_g, 1)):
                        g = self.matrix_g[i][j]
                        y = self.matrix_y[i][j]
                        w = self.matrix_w[k][j]
                        err += (g - y) * g * (-g) * w * f * (1 - f) * x_ext
                v_matrix_new[n][k] = v - (alpha_v * err)
        return v_matrix_new

    def FFNN(self, alpha_v=0.005, alpha_w=0.005, iter_max=5000):
        i = 0
        while i <= iter_max:
            # first Backward propagation 
            # updating matrix_w first as matrix_v depends on 
            self.matrix_w = self.w_correction(alpha_w)
            self.matrix_v = self.v_correction(alpha_v)

            self.matrix_x_extended_2 = self.x_2_matrix()

            self.matrix_f = self.f_matrix()
            self.matrix_f_extended = self.f_1_matrix()

            self.matrix_w = self.w_correction(alpha_w)
            self.matrix_f_extended_2 = self.f_2_matrix()

            self.matrix_g = self.g_matrix()

            err = self.sum_squared_errors_calculation()
            self.error_history.append(err)

            i += 1
        return self.g_matrix()

    def show_error_log(self):
        plt.plot(self.error_history)
        plt.ylabel("Error")
        plt.xlabel("Itteration")
        plt.show()


artificial_neural_network = ANN("data_ffnn_3classes.txt", 3, 3)

artificial_neural_network.FFNN()
artificial_neural_network.show_error_log()


[[1.9643 4.5957]
 [2.2753 3.8589]
 [2.9781 4.5651]
 [2.932  3.5519]
 [3.5772 2.856 ]
 [4.015  3.1937]
 [3.3814 3.4291]
 [3.9113 4.1761]
 [2.7822 4.0431]
 [2.5518 4.6162]
 [3.3698 3.9101]
 [3.1048 3.0709]
 [1.9182 4.0534]
 [2.2638 4.3706]
 [2.6555 3.5008]
 [3.1855 4.2888]
 [3.6579 3.8692]
 [3.9113 3.4291]
 [3.6002 3.1221]
 [3.0357 3.3165]
 [1.5841 3.3575]
 [2.0103 3.2039]
 [1.9527 2.7843]
 [2.2753 2.7127]
 [2.3099 2.9584]
 [2.8283 2.6309]
 [3.0473 2.2931]
 [2.4827 2.0373]
 [2.5057 2.3853]
 [1.8721 2.0577]
 [2.0103 2.3546]
 [1.2269 2.3239]
 [1.8951 2.9174]
 [1.561  3.0709]
 [1.5495 2.6923]
 [1.6878 2.4057]
 [1.4919 2.0271]
 [0.962  2.682 ]
 [1.1693 2.9276]
 [0.8122 2.9992]
 [0.9735 3.3881]
 [1.25   3.1937]
 [1.3191 3.5109]
 [2.2292 2.201 ]
 [2.4482 2.6411]
 [2.7938 1.9656]
 [2.0909 1.6177]
 [2.5403 2.8867]
 [0.9044 3.0198]
 [0.7661 2.5899]
 [0.0864 4.1045]
 [4.6236 2.211 ]
 [4.2382 1.6094]
 [5.2964 1.5612]
 [3.9732 1.2429]
 [4.4475 1.3257]
 [3.6365 2.1778]
 [5.2574 1.5792]
 [4.6737 1.818

ValueError: all the input array dimensions for the concatenation axis must match exactly, but along dimension 1, the array at index 0 has size 1 and the array at index 1 has size 2